# Tahap 1: Impor Library dan Pengaturan Awal

In [ ]:
#!pip install pyswarms

import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T
from transformers import ViTImageProcessor, ViTForImageClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from PIL import Image
from tqdm.auto import tqdm
import pandas as pd
import matplotlib.pyplot as plt

# Library untuk PSO dan visualisasinya
import pyswarms as ps
from pyswarms.utils import plotters as plot

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 3.3 MB/s eta 0:00:00


2025-06-22 08:49:54.614883: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750582194.811755      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750582194.867878      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# --- SEED UNTUK REPRODUCIBILITY ---
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    # Pengaturan ini membuat proses lebih lambat tapi hasilnya konsisten
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

print(f"Random seed diset ke {SEED} untuk hasil yang konsisten.")

Random seed diset ke 42 untuk hasil yang konsisten.


# Tahap 2: Konfigurasi

In [4]:
# --- Konfigurasi Model & Data ---
MODEL_NAME = "google/vit-base-patch16-224"
NUM_CLASSES = 2
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- PATH DATASET (SESUAIKAN JIKA PERLU) ---
BASE_DATA_DIR = "/kaggle/input/cell-images-for-detecting-malaria/cell_images/cell_images/"

# --- Konfigurasi Pencarian PSO ---
# Menggunakan nilai yang lebih ringan dan praktis
N_PARTICLES = 5       # Jumlah partikel
N_ITERATIONS = 5      # Jumlah iterasi
EPOCHS_PER_TRIAL = 3  # Epoch singkat untuk setiap evaluasi

# Opsi PSO menggunakan Konfigurasi Klasik yang disederhanakan
PSO_OPTIONS = {
    'c1': 2.0,
    'c2': 2.0,
    'w': 0.7  # Nilai statis sebagai kompromi dari inersia menurun
}

# Batasan ruang pencarian
MIN_BOUNDS = [1e-6, 16]  # [min_lr, min_batch_size]
MAX_BOUNDS = [1e-3, 64]  # [max_lr, max_batch_size]
BOUNDS = (np.array(MIN_BOUNDS), np.array(MAX_BOUNDS))

print("\n--- Konfigurasi Ditetapkan ---")
print(f"Device: {DEVICE}")
print(f"Model: {MODEL_NAME}")
print(f"PSO: {N_PARTICLES} partikel, {N_ITERATIONS} iterasi, {EPOCHS_PER_TRIAL} epoch per trial")
print(f"Opsi PSO (Klasik): c1={PSO_OPTIONS['c1']}, c2={PSO_OPTIONS['c2']}, w={PSO_OPTIONS['w']}")


--- Konfigurasi Ditetapkan ---
Device: cuda
Model: google/vit-base-patch16-224
PSO: 5 partikel, 5 iterasi, 3 epoch per trial
Opsi PSO (Klasik): c1=2.0, c2=2.0, w=0.7


# Tahap 3: Persiapan Data dan Fungsi Bantu

In [5]:
class MalariaDataset(Dataset):
    """Kelas Dataset Kustom untuk data malaria."""
    def __init__(self, file_paths, labels, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        img_path = self.file_paths[idx]
        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

def snap_to_valid_batch_size(n, values=[16, 32, 64]):
    """Memaksa nilai batch size ke pilihan valid terdekat (pangkat 2)."""
    return min(values, key=lambda x: abs(x - n))

# --- Memuat dan Membagi Data (Dilakukan Sekali) ---
print("\nMemuat dan membagi data...")
parasitized_dir = os.path.join(BASE_DATA_DIR, "Parasitized")
uninfected_dir = os.path.join(BASE_DATA_DIR, "Uninfected")

parasitized_files = [os.path.join(parasitized_dir, f) for f in os.listdir(parasitized_dir) if f.endswith('.png')]
uninfected_files = [os.path.join(uninfected_dir, f) for f in os.listdir(uninfected_dir) if f.endswith('.png')]

all_files = parasitized_files + uninfected_files
all_labels = [0] * len(parasitized_files) + [1] * len(uninfected_files)

# Menggunakan subset 50% data untuk tuning agar proses lebih cepat
_, subset_files, _, subset_labels = train_test_split(
    all_files, all_labels, test_size=0.5, stratify=all_labels, random_state=SEED
)

train_files, val_files, train_labels, val_labels = train_test_split(
    subset_files, subset_labels, test_size=0.2, stratify=subset_labels, random_state=SEED
)
print(f"Data siap: {len(subset_files)} gambar digunakan (Train: {len(train_files)}, Val: {len(val_files)})")

# --- Fungsi Persiapan DataLoader & Transformasi ---
image_processor = ViTImageProcessor.from_pretrained(MODEL_NAME)
target_size = (image_processor.size['height'], image_processor.size['width'])

def prepare_dataloaders(batch_size):
    """Mempersiapkan DataLoader berdasarkan batch size yang diberikan."""
    def vit_core_processor_transform(pil_image, processor):
        return processor(images=pil_image, return_tensors='pt')['pixel_values'].squeeze(0)

    train_transforms = T.Compose([
        T.RandomResizedCrop(target_size, scale=(0.8, 1.0), ratio=(0.75, 1.33)),
        T.RandomRotation(degrees=20), # Tingkatkan sedikit rotasi
        T.ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25, hue=0.12), # Tingkatkan sedikit jitter
        T.RandomHorizontalFlip(),
        T.RandomVerticalFlip(), # Tambahkan vertical flip, bisa relevan untuk sel
        lambda img: vit_core_processor_transform(img, image_processor)
    ])
    val_transforms = T.Compose([
        lambda img: vit_core_processor_transform(img, image_processor)
    ])

    train_dataset = MalariaDataset(train_files, train_labels, transform=train_transforms)
    val_dataset = MalariaDataset(val_files, val_labels, transform=val_transforms)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    return train_loader, val_loader


Memuat dan membagi data...
Data siap: 13779 gambar digunakan (Train: 11023, Val: 2756)


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

# Tahap 4: Fungsi Objektif PSO

In [6]:
def objective_function(particles):
    """Mengevaluasi setiap partikel dan mengembalikan cost-nya."""
    num_particles = particles.shape[0]
    costs = []

    for i, p in enumerate(particles):
        learning_rate = p[0]
        # Menggunakan fungsi bantu untuk batch size
        batch_size = snap_to_valid_batch_size(p[1])

        print(f"\n[Trial Partikel {i+1}/{num_particles}] LR: {learning_rate:.1E}, BS: {batch_size}")

        try:
            model = ViTForImageClassification.from_pretrained(
                MODEL_NAME, num_labels=NUM_CLASSES, ignore_mismatched_sizes=True
            ).to(DEVICE)

            train_loader, val_loader = prepare_dataloaders(batch_size)
            optimizer = optim.Adam(model.parameters(), lr=learning_rate)
            criterion = nn.CrossEntropyLoss()
            best_val_accuracy_in_trial = 0.0

            for epoch in range(EPOCHS_PER_TRIAL):
                model.train()
                for inputs, labels in train_loader:
                    inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                    optimizer.zero_grad()
                    outputs = model(pixel_values=inputs).logits
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()

                model.eval()
                val_preds_all, val_labels_all = [], []
                with torch.no_grad():
                    for inputs, labels in val_loader:
                        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                        outputs = model(pixel_values=inputs).logits
                        _, preds = torch.max(outputs, 1)
                        val_preds_all.extend(preds.cpu().numpy())
                        val_labels_all.extend(labels.cpu().numpy())

                epoch_val_accuracy = accuracy_score(val_labels_all, val_preds_all)
                if epoch_val_accuracy > best_val_accuracy_in_trial:
                    best_val_accuracy_in_trial = epoch_val_accuracy

            cost = 1.0 - best_val_accuracy_in_trial
            print(f"  > Selesai. Best Val Acc: {best_val_accuracy_in_trial:.4f} -> Cost: {cost:.4f}")

        except Exception as e:
            print(f"  > ERROR: {e}. Memberikan penalti.")
            cost = 1.0

        costs.append(cost)
    return np.array(costs)

# Tahap 5: Eksekusi dan Visualisasi PSO

In [7]:
if __name__ == "__main__":
    print("\n" + "="*50)
    print("--- MEMULAI OPTIMASI HYPERPARAMETER DENGAN PSO ---")
    print("="*50)

    optimizer = ps.single.GlobalBestPSO(
        n_particles=N_PARTICLES,
        dimensions=len(MIN_BOUNDS),
        options=PSO_OPTIONS,
        bounds=BOUNDS
    )

    best_cost, best_pos = optimizer.optimize(objective_function, iters=N_ITERATIONS)

    # --- Menampilkan Hasil Akhir ---
    print("\n\n" + "="*50)
    print("--- OPTIMASI PSO SELESAI ---")
    print("="*50)

    # Tampilkan tabel riwayat untuk analisis yang lebih dalam
    results_data = []
    for i in range(N_ITERATIONS):
        for j in range(N_PARTICLES):
            lr = optimizer.pos_history[i][j][0]
            bs = snap_to_valid_batch_size(optimizer.pos_history[i][j][1])
            cost = optimizer.cost_history[i] # Cost history adalah best cost per iterasi
        results_data.append({
            'iterasi': i + 1,
            'best_cost_iterasi_ini': optimizer.cost_history[i],
            'best_lr_saat_ini': optimizer.pos_history[i][np.argmin(optimizer.cost_history)][0],
            'best_bs_saat_ini': snap_to_valid_batch_size(optimizer.pos_history[i][np.argmin(optimizer.cost_history)][1])
        })
    results_df = pd.DataFrame(results_data)
    print("\n--- Ringkasan Hasil per Iterasi ---")
    print(results_df.to_string(index=False))

    best_accuracy = 1.0 - best_cost
    print(f"\nAkurasi validasi tertinggi yang dicapai: {best_accuracy:.4f}")

    print("\n>>> HYPERPARAMETER TERBAIK YANG DITEMUKAN <<<")
    print("--------------------------------------------------")
    print(f"  Learning Rate: {best_pos[0]:.6f}")
    print(f"  Batch Size: {snap_to_valid_batch_size(best_pos[1])}")
    print("--------------------------------------------------")


    # --- Visualisasi ---
    print("\n--- Visualisasi Proses Optimasi ---")
    
    # 1. Plot Cost History
    # Memanggil fungsi menggunakan plot.plot_cost
    plot.plot_cost(optimizer.cost_history)
    plt.title("Perkembangan Cost (1 - Akurasi) per Iterasi", fontsize=14)
    plt.xlabel("Iterasi", fontsize=12)
    plt.ylabel("Best Cost", fontsize=12)
    plt.grid(True)
    plt.show()

    # 2. Plot Contour
    try:
        plot.plot_contour(pos_history=optimizer.pos_history,
                         title="Pergerakan Partikel di Ruang Pencarian",
                         designer=None)
        plt.xlabel("Learning Rate", fontsize=12)
        plt.ylabel("Batch Size", fontsize=12)
        plt.grid(True)
        plt.show()
    except Exception as e:
        print(f"\nTidak dapat membuat plot kontur: {e}")

2025-06-22 08:52:08,320 - pyswarms.single.global_best - INFO - Optimize for 5 iters with {'c1': 2.0, 'c2': 2.0, 'w': 0.7}



--- MEMULAI OPTIMASI HYPERPARAMETER DENGAN PSO ---


pyswarms.single.global_best:   0%|          |0/5


[Trial Partikel 1/5] LR: 6.1E-04, BS: 16


config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

...

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  > Selesai. Best Val Acc: 0.9499 -> Cost: 0.0501

[Trial Partikel 5/5] LR: 9.2E-05, BS: 32


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
pyswarms.single.global_best: 100%|██████████|5/5, best_cost=0.0287
2025-06-22 13:29:16,559 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.028664731494920215, best pos: [9.19422509e-05 4.27585879e+01]


  > Selesai. Best Val Acc: 0.9713 -> Cost: 0.0287


--- OPTIMASI PSO SELESAI ---

--- Ringkasan Hasil per Iterasi ---
 iterasi  best_cost_iterasi_ini  best_lr_saat_ini  best_bs_saat_ini
       1               0.035196          0.000593                16
       2               0.035196          0.000999                32
       3               0.033019          0.000950                64
       4               0.030479          0.000379                64
       5               0.028665          0.000092                32

Akurasi validasi tertinggi yang dicapai: 0.9713

>>> HYPERPARAMETER TERBAIK YANG DITEMUKAN <<<
--------------------------------------------------
  Learning Rate: 0.000092
  Batch Size: 32
--------------------------------------------------

--- Visualisasi Proses Optimasi ---


AttributeError: module 'pyswarms.utils.plotters' has no attribute 'plot_cost'